# **DEPENDENCIES FOR COLAB AND UI**

In [ ]:
!pip install import_ipynb
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import *
import import_ipynb

# **PDF To Image**

In [ ]:
#move to current directory

/content/drive/My Drive/Colab Notebooks
digital_processing.ipynb  pdfconvert.ipynb	   text_extraction.ipynb
flask.ipynb		  table_extraction1.ipynb
mlassign.ipynb		  tabletest.ipynb


In [ ]:
import pdfconvert
"""
pdfconvert
  This module is for converting pdf to image page by page
  For more info, refer to pdfconvert.py
"""

'\npdfconvert\n  This module is for converting pdf to image page by page\n  For more info, refer to pdfconvert.py\n'

In [ ]:
!pip install PyPDF2
import PyPDF2


In [ ]:
def pdfsection(filepath):
  """
  This module is used to convet pd to image
  Parameters:  
            String : Path to the input pdf
"""
  pdfFileObj = open(filepath,"rb")
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  numPages = pdfReader.numPages 
  print (numPages)
  imgpath = "output/dataset1/"
  pdfconvert.convert2pdf(filepath,imgpath)
  return numPages

# **TABLE EXTRACTION**

In [ ]:
import table_extraction1
import cv2
"""
table_extraction1
  This module is for extracting table.
  For more info, refer to table_extraction1.py
"""


'\ntable_extraction1\n  This module is for extracting table.\n  For more info, refer to table_extraction1.py\n'

In [ ]:
def tablesection(numPages):
  """
  This module is for extracting table from input image.
  Parameters:  
            int : The number of pages in invoice.
"""
  for i in range(1,numPages+1):
    srcpath = "output/dataset1/"+str(i)+'.jpg'#img from sourcepath to excel in excelpath
    excelpath = "output/dataset1/excel"+str(i)+".xlsx"
    croppath = "output/dataset1/cropped"+str(i)+".jpg"
    destpath = "output/dataset1/desttable"+str(i)+".png"  
    cropped,data = table_extraction1.textExtraction(srcpath,excelpath,destpath)
    cv2.imwrite(croppath,cropped)
  

# **DIGITAL IMAGE PROCESSING**

In [ ]:
import digital_processing
"""
digital_processing
  This module is for applying threshold and processing image.
  For more info, refer to digital_processing.py
"""

'\ndigital_processing\n  This module is for applying threshold and processing image.\n  For more info, refer to digital_processing.py\n'

# **TEXT EXTRACTION**

In [ ]:
import text_extraction
"""
text_extraction
  This module is for extracting text from  cropped image.
  For more info, refer to text_extraction.py
"""

'\ntext_extraction\n  This module is for extracting text from  cropped image.\n  For more info, refer to text_extraction.py\n'

In [ ]:

def textsection(numPages):
  """
  This module is for extracting text from  cropped image.
  Parameters:  
            int : The number of pages in invoice.
"""
  for i in range(1,numPages+1):
    imgpath = "output/dataset1/cropped"+str(i)+".jpg"
    digital_processing.threshAndConvert(imgpath)
    destpath = "output/dataset1/excel"+str(i)+".png"
    textimg = "output/dataset1/excel"+str(i)+".png"
    #print(i)
    header = text_extraction.formatText(imgpath,textimg,destpath)
    print(header)
  

  

# **DOWNLOAD INVOICE ASSETS**

In [ ]:
import shutil
import os

In [ ]:
def downloadAll(dirpath,output):
  shutil.make_archive(output, 'zip', dirpath)
  return output

#downloadAll("/content/drive/My Drive/hack_space/output/dataset1","/content/drive/My Drive/hack_space/output/dt1")

# **START E-INVOICE WEB-SERVER**

In [ ]:
app = Flask(__name__,template_folder="templates")
app.secret_key = os.urandom(24)
app._static_folder = "static"
run_with_ngrok(app)   #starts ngrok when the app is run
numPages = 0

@app.route('/', defaults={'convertbutton': "true",'downloadbutton': "false"})
@app.route("/<convertbutton>/<downloadbutton>")
def home(convertbutton,downloadbutton):
    return render_template("index.html",convertbutton=convertbutton,downloadbutton=downloadbutton)

@app.route("/upload", methods = ['POST'])  
def upload():
  print("entered")
  if request.method == 'POST':
    print("it is upload post")
    if 'file' not in request.files:
      print("file nahi hey")
    else:
      print("iruku")
      file = request.files['file']
      print("file is there! "+file.filename)
      filepath = "output/dataset1/invoice.pdf"
      file.save(filepath)
      print("file saved "+filepath)
      numPages = pdfsection(filepath)
      tablesection(numPages)
      print("upload ends")
    return redirect(url_for("home",convertbutton="false",downloadbutton="false"))  

@app.route('/download/<path:site>/<convertbutton>/<downloadbutton>')
def download(site,convertbutton,downloadbutton):
  print(site)
  url = '/'+site
  print(url)
  return render_template("index.html",site=url,convertbutton=convertbutton,downloadbutton=downloadbutton)


@app.route('/convert', methods = ['POST'])  
def convert():  
    if request.method == 'POST':
      
      textsection(numPages)
      site = downloadAll("output/dataset1","output/dt1")
      site = site[1:]
      print('Successfully Converted')
      return redirect(url_for("download",site=site,convertbutton="true",downloadbutton="true"))


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4f47374434e4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Aug/2020 14:47:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 14:47:11] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 14:47:11] "GET /static/invoice.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2020 14:47:13] "GET /favicon.ico HTTP/1.1" 404 -


# **RUN PYTHON**